In [1]:
pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [3]:
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.max_rows', None)     # Afficher toutes les lignes
pd.set_option('display.max_colwidth', None) # Afficher la largeur de colonne sans tronquer
pd.set_option('display.width', None) 

In [4]:
# url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20241008-071041/valeursfoncieres-2023.txt.zip"

url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2024/full.csv.gz"
# Envoyer une requête HTTP pour obtenir le fichier CSV

downloaded_file = "full.csv.gz"
response = rq.get(url)

with open(downloaded_file, 'wb') as file:
    file.write(response.content)

# Décompresser le fichier
with gzip.open(downloaded_file, 'rb') as f_in:
    with open("full.csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Charger le fichier CSV dans un DataFrame
df = pd.read_csv("full.csv")

# Afficher les premières lignes du DataFrame
print(df.head(20))

# Optionnel : supprimer le fichier compressé après décompression
os.remove(downloaded_file)

C:\Users\flori\AppData\Local\Temp\ipykernel_30044\883604032.py:18: DtypeWarning: Columns (8,10,12,14,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full.csv")


   id_mutation date_mutation  numero_disposition nature_mutation  \
0       2024-1    2024-01-02                   1           Vente   
1       2024-2    2024-01-03                   2           Vente   
2       2024-3    2024-01-08                   1           Vente   
3       2024-4    2024-01-03                   1           Vente   
4       2024-4    2024-01-03                   1           Vente   
5       2024-4    2024-01-03                   1           Vente   
6       2024-4    2024-01-03                   1           Vente   
7       2024-5    2024-01-09                   1           Vente   
8       2024-5    2024-01-09                   1           Vente   
9       2024-5    2024-01-09                   1           Vente   
10      2024-5    2024-01-09                   1           Vente   
11      2024-5    2024-01-09                   1           Vente   
12      2024-5    2024-01-09                   1           Vente   
13      2024-5    2024-01-09                   1

In [ ]:
# print(df.head())
# print(df.shape)
# print(f"Nombre de lignes : {n_lignes}")
# print(f"Nombre de colonnes : {n_colonnes}")
print(df.head())
nan_info = pd.DataFrame({
    'Type': df.dtypes,
    'NaN_count': df.isna().sum(),
    'NaN_ratio': df.isna().mean()
})

# modalites = df['Type de voie'].value_counts(normalize=True) * 100
# print(modalites)

print(nan_info)

  id_mutation date_mutation  numero_disposition nature_mutation  \
0      2024-1    2024-01-02                   1           Vente   
1      2024-2    2024-01-03                   2           Vente   
2      2024-3    2024-01-08                   1           Vente   
3      2024-4    2024-01-03                   1           Vente   
4      2024-4    2024-01-03                   1           Vente   

   valeur_fonciere  adresse_numero adresse_suffixe adresse_nom_voie  \
0            346.5             NaN             NaN       LE DELIVRE   
1          10000.0             NaN             NaN   CHEVRY DESSOUS   
2         249000.0             NaN             NaN       PIN HAMEAU   
3         329500.0            29.0             NaN       PL DU JURA   
4         329500.0            29.0             NaN       PL DU JURA   

  adresse_code_voie  code_postal code_commune nom_commune code_departement  \
0              B020       1230.0         1076      Chaley                1   
1             

In [9]:
## Affiche la première ligne et et toutes les variables
premiere_ligne = df.iloc[0].T
print(premiere_ligne)

id_mutation                             2024-1
date_mutation                       2024-01-02
numero_disposition                           1
nature_mutation                          Vente
valeur_fonciere                          346.5
adresse_numero                             NaN
adresse_suffixe                            NaN
adresse_nom_voie                    LE DELIVRE
adresse_code_voie                         B020
code_postal                             1230.0
code_commune                              1076
nom_commune                             Chaley
code_departement                             1
ancien_code_commune                        NaN
ancien_nom_commune                         NaN
id_parcelle                     010760000B0514
ancien_id_parcelle                         NaN
numero_volume                              NaN
lot1_numero                                NaN
lot1_surface_carrez                        NaN
lot2_numero                                NaN
lot2_surface_

In [6]:
# 1er filtre : Supprimer toutes les transactions NaN
df = df.dropna(subset=['valeur_fonciere'])

print(df['valeur_fonciere'].isna().sum())


0


In [11]:
colonnes_a_supprimer = ['ancien_code_commune',
                        'ancien_nom_commune',
                        'ancien_id_parcelle',
                        'numero_volume',
                        'code_nature_culture_speciale',
                        'nature_culture_speciale',
                        'lot1_numero',
                        'lot2_numero',
                        'lot3_numero',
                        'lot4_numero',
                        'lot5_numero',
                        'lot1_surface_carrez',
                        'lot2_surface_carrez',
                        'lot3_surface_carrez',
                        'lot4_surface_carrez',
                        'lot5_surface_carrez'
                        ]

df.drop(columns=colonnes_a_supprimer, inplace=True)

In [24]:
print(df['adresse_numero'].dtype)
print(df['adresse_nom_voie'].dtype)
print(df['code_postal'].dtype)
print(df['nom_commune'].dtype)

df['adresse_numero'] = df['adresse_numero'].astype('string')
df['adresse_numero'] = df['adresse_numero'].str.replace('.0', '', regex=False)
df['adresse_numero'] = df['adresse_numero'].str.replace('nan', '', regex=False)
df['adresse_numero'] = df['adresse_numero'].fillna('')


# Ajouter un 0 devant si le code postal a 4 caractères
df['code_postal'] = df['code_postal'].astype('string')
df['code_postal'] = df['code_postal'].str.replace('.0', '', regex=False)
df['code_postal'] = df['code_postal'].str.replace('nan', '', regex=False)
df['code_postal'] = df['code_postal'].apply(lambda x: x.zfill(5))

string
object
object
object


In [49]:
# Compléter par le type de voie

voie = pd.read_csv('voie.csv',sep=";")
print(voie.head())

# Liste des abréviations de types de voie
abbreviations = voie['abreviation'].tolist()

# Fonction pour extraire et vérifier la première chaîne avant le premier espace
def check_abbreviation(adresse):
    if isinstance(adresse, str):
        # Extraire la première partie avant l'espace
        parts = adresse.split(' ')
        first_word = parts[0]
        # Vérifier si elle appartient à la liste des abréviations
        if first_word in abbreviations:
            # Si le premier mot est une abréviation, extraire le reste de l'adresse
            nom_voie = ' '.join(parts[1:])  # Tout ce qui suit le premier mot
        else:
            # Si ce n'est pas une abréviation, mettre tout l'adresse dans nom_voie
            nom_voie = adresse
            first_word = ' '  # Remplacer par un espace pour le premier mot
    else:
        first_word = ' '
        nom_voie = ''
    
    return first_word, nom_voie

# Appliquer la fonction à la colonne 'adresse_nom_voie'
result = df['adresse_nom_voie'].apply(check_abbreviation)

# Décomposer les résultats dans les colonnes 'type_voie' et 'nom_voie'
df['type_voie'] = result.apply(lambda x: x[0])  # Prend le premier élément
df['nom_voie'] = result.apply(lambda x: x[1])   # Prend le deuxième élément

df = df.merge(voie,left_on=['type_voie'],right_on=['abreviation'])

  abreviation type_voie_complet
0         RUE               Rue
1          AV            Avenue
2         RTE             Route
3         CHE            Chemin
4          BD         Boulevard


In [ ]:
# print(df['No voie'].dtype)
# print(df['Type de voie'].dtype)
# print(df['Voie'].dtype)
# print(df['Code postal'].dtype)
df['Adresse'] = df['adresse_numero'] + ' ' + df['type_voie_complet'] + ' ' + df['nom_voie'] + ' ' + df['code_postal'] + ' ' + df['nom_commune']

print(df['Adresse'].head(20))

   id_mutation date_mutation  numero_disposition nature_mutation  \
0       2024-4    2024-01-03                   1           Vente   
1       2024-4    2024-01-03                   1           Vente   
2       2024-4    2024-01-03                   1           Vente   
3       2024-4    2024-01-03                   1           Vente   
4       2024-7    2024-01-03                   1           Vente   
5       2024-7    2024-01-03                   1           Vente   
6       2024-7    2024-01-03                   1           Vente   
7       2024-7    2024-01-03                   1           Vente   
8       2024-8    2024-01-03                   1           Vente   
9      2024-10    2024-01-05                   2           Vente   
10     2024-11    2024-01-03                   1           Vente   
11     2024-12    2024-01-03                   2           Vente   
12     2024-13    2024-01-05                   1           Vente   
13     2024-15    2024-01-08                   1

In [26]:
# Combien de transactions ne sont pas géolocalisées ?

# Compter les lignes avec NaN ou des chaînes vides dans 'latitude' et 'longitude'
latitude_missing = df['latitude'].isna().sum() + (df['latitude'] == '').sum()
longitude_missing = df['longitude'].isna().sum() + (df['longitude'] == '').sum()

print(f"Nombre de lignes avec 'latitude' vide ou NaN : {latitude_missing}")
print(f"Nombre de lignes avec 'longitude' vide ou NaN : {longitude_missing}")

Nombre de lignes avec 'latitude' vide ou NaN : 20855
Nombre de lignes avec 'longitude' vide ou NaN : 20855


In [60]:
# Filtrer les lignes avec 'latitude' ou 'longitude' vide ou NaN, mais avec 'Adresse' non vide
df_missing_lat = df[
    ((df['latitude'].isna()) | (df['latitude'] == '') |
     (df['longitude'].isna()) | (df['longitude'] == '')) &
    (df['Adresse'] != '') & (~df['Adresse'].isna())
]
print(df_missing_lat.shape[0])
adresses_missing = df_missing_lat['Adresse'].drop_duplicates().reset_index(drop=True)
adresses_missing = pd.DataFrame(adresses_missing, columns=['Adresse'])

print(adresses_missing.shape[0])
print(df_missing_lat.head())
# Compter le nombre de lignes correspondant au filtre
count = df_missing_lat.shape[0]

print(f"Nombre de lignes avec 'latitude' ou 'longitude' vide/NaN et une 'Adresse' renseignée : {count}")

9585
3982
     id_mutation date_mutation  numero_disposition nature_mutation  \
38       2024-34    2024-01-03                   1           Vente   
39       2024-34    2024-01-03                   1           Vente   
40       2024-34    2024-01-03                   1           Vente   
1947   2024-1409    2024-02-29                   1           Vente   
3049   2024-2133    2024-02-23                   1           Vente   

      valeur_fonciere adresse_numero adresse_suffixe     adresse_nom_voie  \
38           187584.0             47             NaN    CHE DE FONTANETTE   
39           187584.0             47             NaN    CHE DE FONTANETTE   
40           187584.0             47             NaN    CHE DE FONTANETTE   
1947          50000.0             11             NaN  AV DES SABLONNIERES   
3049         323500.0             10             NaN  RTE DE SAINT MARCEL   

     adresse_code_voie code_postal code_commune         nom_commune  \
38                0016       01300 

In [ ]:
# Géolocalisation des adresses manquantes


In [ ]:
# Merge région

region = pd.read_csv('code_region.csv', sep=';')
print(region.head())
df = df.merge(region,left_on=["Code departement"],right_on=['regionCode'])

In [53]:
# Initialiser le géocodeur
geolocator = Nominatim(user_agent="geoapiExercises")

# Fonction pour obtenir la latitude et la longitude
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return math.nan, math.nan  # Retourne NaN si la localisation n'est pas trouvée
    except GeocoderTimedOut:
        return get_lat_long(address)  # Refaire la demande si un timeout se produit
    except Exception as e:
        return math.nan, math.nan  # Retourne NaN pour d'autres exceptions


In [62]:
print(adresses_missing.head())

adresses_missing['lat'], adresses_missing['long'] = zip(*adresses_missing['Adresse'].apply(get_lat_long))

print(adresses_missing[['Adresse','lat','long']].head())

                                            Adresse
0  47 Chemin DE FONTANETTE 01300 Murs-et-Gélignieux
1  11 Avenue DES SABLONNIERES 01210 Ferney-Voltaire
2          10 Route DE SAINT MARCEL 01390 Monthieux
3    12 Rue ANTOINE BELAYSOUD 01000 Bourg-en-Bresse
4    14 Rue ANTOINE BELAYSOUD 01000 Bourg-en-Bresse


KeyboardInterrupt: 